# Импорт библиотек

In [1]:
import pandas as pd

# А.Работа с данными

## А.1 Загрузка данных

In [3]:
df = pd.read_csv('../data/pokemon.csv')

## А.2 Краткий обзор данных

In [4]:
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


## А.3 Предобработка данных

### А.3.1 Обработка пропущенных значений

In [ ]:
titanic_df.isnull().sum()

#### А.3.1.1 k-NN Imputer

In [ ]:
# https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e
# https://datasciencebeginners.com/2018/11/07/visualization-of-imputed-values-using-vim/ - visualization

#### А.3.1.2 Глубокое обучение

In [ ]:
import datawig

df_train, df_test = datawig.utils.random_split(train)

# Инициализируем модель SimpleImputer
imputer = datawig.SimpleImputer(
    input_columns=['1','2','3','4','5','6','7', 'target'], # из каких столбцов брать информацию для импутации
    output_column= '0', # какой столбец восстанавливаем
    output_path = 'imputer_model' # куда записывать модель и её метрики
    )

# Тренируем модель
imputer.fit(train_df=df_train, num_epochs=50)

# Проводим импутацию и возвращаем восстановленный набор данных
imputed = imputer.predict(df_test)

#### А.3.1.3 Удаление записи

In [ ]:
titanic_df.dropna()

#### А.3.1.4 Удаление признака

In [ ]:
df = df.drop(['feature'], axis=1) or titanic_df.dropna(axis=1)

#### А.3.1.5 Заполнение самым частым значением

In [ ]:
df.fillna(df['name'].value_counts().index[0])

#### А.3.1.6 Заполнение самым средним или медианой

In [ ]:
df.fillna(titanic_df.mean()) or df.fillna(titanic_df.median())

### А.3.2 Обработка признаков

#### А.3.2.2 Обработка категориальных признаков

In [ ]:
# numeric
def RoseOfWind(data, col):
  data[col] = data[col].replace("N"  ,  1)
  data[col] = data[col].replace("NNE",  2)
  data[col] = data[col].replace("NE" ,  3)
  data[col] = data[col].replace("ENE",  4)
  data[col] = data[col].replace("E"  ,  5)
  data[col] = data[col].replace("ESE",  6)
  data[col] = data[col].replace("SE" ,  7)
  data[col] = data[col].replace("SSE",  8)
  data[col] = data[col].replace("S"  ,  9)
  data[col] = data[col].replace("SSW", 10)
  data[col] = data[col].replace("SW" , 11)
  data[col] = data[col].replace("WSW", 12)
  data[col] = data[col].replace("W"  , 13)
  data[col] = data[col].replace("WNW", 14)
  data[col] = data[col].replace("NW" , 15)
  data[col] = data[col].replace("NNW", 16)

RoseOfWind(data, 'WindGustDir')
RoseOfWind(data, 'WindDir9am')
RoseOfWind(data, 'WindDir3pm')

#кодируем признаки с помощью метода One-Hot
type_1_dummies = pd.get_dummies(pokemon_df['Type 1'])
type_2_dummies = pd.get_dummies(pokemon_df['Type 2'])
types_df = pd.DataFrame(index = pokemon_df.index)
types = list(type_2_dummies.columns.values)
for t in types:
    types_df[t] = type_1_dummies[t] + type_2_dummies[t]

pokemon_df = pd.concat([pokemon_df, types_df], sort = False, axis = 1)

#удаляем столбцы Type 1 и Type 2

pokemon_df.drop(["Type 1","Type 2"], axis = 1, inplace = True)

#### А.3.2.2 Обработка порядковых признаков

In [ ]:
# same lil nigga

#### А.3.2.3 Обработка бинарных признаков

In [ ]:
pokemon_df.Legendary.replace({True:1,False:0}, inplace = True)

#### А.3.2.4 Обработка вещественных признаков

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df.iloc[:,0:7])
scaled_df = pd.DataFrame(np.array(scaler.transform(df.iloc[:,0:7])), columns = ["Total","HP","Attack","Defense","Sp_Atk","Sp_Def","Speed"], index = df.index)


df = pd.concat([scaled_df, df.iloc[:,7:]], axis = "columns")

### А.3.3 Обработка выбросов

In [ ]:
# identify outliers in the training dataset
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X_train)

# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]

# evaluate model performance with outliers removed using isolation forest
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)
# retrieve the array
data = df.values
# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
# identify outliers in the training dataset
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

#https://towardsdatascience.com/anomaly-detection-with-isolation-forest-visualization-23cd75c281e2 - visualization

### А.3.4 Учет корреляции

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(dataframe.corr())
plt.show()

# heat map

## А.4 Визуализация данных

### А.4.1 Heat map

### А.4.2 Pair plot

### А.4.3 Histogram

### А.4.4 Pie chart

### А.4.5 Graphic

# B.Работа с моделями

## Отбор признаков

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

array = data.values
X = array[:,0:24]
Y = array[:,24]
model = ExtraTreesClassifier()
model.fit(X, Y)
print(model.feature_importances_)

id = []
for col in data.columns:
    id.append(col)
id.pop(24)
feat = model.feature_importances_.tolist()
ff = pd.DataFrame()
ff['Признак'] = id
ff['Важность признака'] = feat
ff.sort_values(by=['Важность признака'], ascending = False)

idNum = []
corrArr = []
for i in range(24):
    idNum.append(i)
for col in data.columns:
    corrArr.append(np.corrcoef(data[col].values, data['RainTomorrow'].values)[0][1])
corrArr.pop(24)

fig, ax = plt.subplots(tight_layout=True)
hist = ax.hist2d(idNum, corrArr)

fig, axs = plt.subplots()
axs.hist(corrArr)

## Разбиение выборки

In [ ]:
#создадим датафреймы X и y
y = pokemon_df.Legendary
pokemon_df = pokemon_df.drop(['Legendary'], axis=1)
X = pokemon_df

from sklearn.model_selection import train_test_split
#разбиваем всю выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 2)

## Модель "название модели"

### Построение модели

In [ ]:
clf = GradientBoostingClassifier()

### Обучение модели без гиперпараметров

In [ ]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

### Обучение модели с гиперпараметрами

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

parameters = {
    "n_estimators":[5,50,250,500],
    "max_depth":[1,3,5,7,9],
    "learning_rate":[0.01,0.1,1,10,100]
}


grid_search = GridSearchCV(estimator = clf, param_grid = parameters, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)
print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

### Оценка моделей

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# C.Заключительная часть

## Сериализация моделей

In [ ]:
model = pickle.dumps(Bagging)
Bagging_byte = pickle.loads(model)
with open('Bagging.pkl', 'wb') as output:
       pickle.dump(Bagging, output)

In [ ]:
model = pickle.dumps(GrBosstClass)
GrBosstClass_byte = pickle.loads(model)
with open('GrBosstClass.pkl', 'wb') as output:
       pickle.dump(GrBosstClass, output)

## Выводы

## Streamlit

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from joblib import dump, load
from sklearn import ensemble
from sklearn import metrics
import numpy as np


def main():
  GrBosstClass = load_GRB()
  Bagging = load_Bagging()
  data = loadSet()
  defaulData = loadDefaultSet()
  currentModelSelected = 0
  vl15H = 0
  speedWind = 0
  cloydy3H = 0
  vl9H = 0
  rainToday = 1
  sun = 0
  tempAir15H = 0
  dav15H = 0
  minTemp = 0
  maxTemp = 0
  
  st.title("Модуль С")
  st.markdown("<style> .big-font {font-size:30px !important; }</style>", unsafe_allow_html=True)
  st.sidebar.markdown('<p class="big-font"> <b>Модели</b> </p>', unsafe_allow_html=True)
  modelSelect = st.sidebar.radio("Выберите модель машинного обучения", ["Градиентный бустинг", "Бэггиннг"])

  st.sidebar.markdown('<p class="big-font"> <b>Дата сеты</b> </p>', unsafe_allow_html=True)
  dataSetSelect = st.sidebar.radio("Выберите дата сет", ["Исходный", "После препроцесса"])

  if dataSetSelect == "После препроцесса":
    '''В датасете после препроцесса остались только самые важные десять полей. Также перекодированы столбцы: локация, направление ветра и был ли сегодня дождь.'''
    st.write(data.head(10))

  if dataSetSelect == "Исходный":
    ''' Исходный датасет имеет полный список полей. Также в нём не заполнены пустые значения и ни один признак не перекодирован.'''
    st.write(defaulData.head(10))
      

  if modelSelect == "Градиентный бустинг":
    currentModelSelected = GrBosstClass
    "Предсказание градиентного бустинга"
    "Градиентный бустинг предсказывает с точностью в 85%"


  if modelSelect == "Бэггиннг":
    currentModelSelected = Bagging
    "Предсказание бэггиннга"
    "Бэггинг предсказывает с точностью в 84%"
  "Для предсказания дождя на завтрашний день исопльзуются основные 10 параметров."
  ""  
  inpType = st.radio("Как будете вводить данные?", ['Строкой', "Буду заполнять каждое поле отдельно!"])
  if inpType == "Буду заполнять каждое поле отдельно!":
    vl15H = st.number_input("Влажность воздуха в 15:00")
    speedWind = st.number_input("Скорость порыва ветра")
    cloydy3H = st.number_input("Облачность в 15:00 (от 0 до 9)")
    vl9H = st.number_input("Влажность в 9 утра")
    rainTodayStr = st.radio("Был ли сегодня дождь?", ["Да" , "Нет"])
    rainToday = 1
    if rainTodayStr =="Да":
      rainToday = 1
    else:
      rainToday = 0
    sun = st.number_input("Солнечный свет (от 0 до 13,9)")
    tempAir15H = st.number_input("Температура воздуха в 15:00")
    dav15H = st.number_input("Давление в 15:00")
    minTemp = st.number_input("Минимальная температура воздуха")
    maxTemp = st.number_input("Максимальная температура воздуха")
  if inpType == "Строкой":
    a = st.text_input('Ввести данные строкой:', help= "Ввдетие числа через запятую. Без точек и запятых в конце строки!")
    if a:
      a = a.split(',')
      a = [float(i) for i in a]
      polya = a
  else:
    polya = int(currentModelSelected.predict([[vl15H,speedWind,cloydy3H,vl9H,rainToday,sun,tempAir15H,dav15H,minTemp,maxTemp]]))
  "Введенные данные: " + str(vl15H) + ", " + str(speedWind) + ", " + str(cloydy3H) + ", " + str(vl9H) + ", " +str(rainToday) + ", " + str(sun) + ", " + str(tempAir15H) + ", " + str(dav15H) + ", " + str(minTemp) + ", " + str(maxTemp) + "."
  if st.button("Рассчитать прогноз", key=None, help="Предсказать будет ли дождь завтра при помощи метода " + modelSelect):
    res = polya 
    if res == 0:
      st.markdown("<p style=\"font-size:30px\">Скорее всего завтра не будет дождя &#128516;</p>", unsafe_allow_html=True)
    else:
      st.markdown("<p style=\"font-size:30px\">Скорее всего завтра будет дождь... &#128532;</p>", unsafe_allow_html=True)
  
@st.cache
def loadSet():
  df = pd.read_csv("data/watherPreProcces.csv")
  return df

### models

@st.cache
def load_GRB():
  with open('models/GrBosstClass.pkl', 'rb') as pkl_file:
    grb = pickle.load(pkl_file)
  return grb


@st.cache(allow_output_mutation=True)
def load_Bagging():
  with open('models/Bagging.pkl', 'rb') as pkl_file:
    Bagging = pickle.load(pkl_file)
  return Bagging

@st.cache
def loadDefaultSet():
  df = pd.read_csv("data/weather.csv")
  return df

if __name__ == "__main__":
  main()
